# Q2: Data Cleaning

**Phase 3:** Data Cleaning & Preprocessing  
**Points: 9 points**

**Focus:** Handle missing data, outliers, validate data types, remove duplicates.

**Lecture Reference:** Lecture 11, Notebook 1 ([`11/demo/01_setup_exploration_cleaning.ipynb`](https://github.com/christopherseaman/datasci_217/blob/main/11/demo/01_setup_exploration_cleaning.ipynb)), Phase 3. Also see Lecture 05 (data cleaning).

---

## Setup

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Load data from Q1 (or directly from source)
df = pd.read_csv('data/beach_sensors.csv')
# If you saved cleaned data from Q1, you can load it:
# df = pd.read_csv('output/q1_exploration.csv')  # This won't work - load original

---

## Objective

Clean the dataset by handling missing data, outliers, validating data types, and removing duplicates.

**Time Series Note:** For time series data, forward-fill (`ffill()`) is often appropriate for missing values since sensor readings are continuous. However, you may choose other strategies based on your analysis.

---

## Required Artifacts

You must create exactly these 3 files in the `output/` directory:

### 1. `output/q2_cleaned_data.csv`
**Format:** CSV file
**Content:** Cleaned dataset with same structure as original (same columns)
**Requirements:**
- Same columns as original dataset
- Missing values handled (filled, dropped, or imputed)
- Outliers handled (removed, capped, or transformed)
- Data types validated and converted
- Duplicates removed
- **Sanity check:** Dataset should retain most rows after cleaning (at least 1,000 rows). If you're removing more than 50% of data, reconsider your strategy—imputation is usually preferable to dropping rows for this dataset.
- **No index column** (save with `index=False`)

### 2. `output/q2_cleaning_report.txt`
**Format:** Plain text file
**Content:** Detailed report of cleaning operations
**Required information:**
- Rows before cleaning: [number]
- Missing data handling method: [description]
  - Which columns had missing data
  - Method used (drop, forward-fill, impute, etc.)
  - Number of values handled
- Outlier handling: [description]
  - Detection method (IQR, z-scores, domain knowledge)
  - Which columns had outliers
  - Method used (remove, cap, transform)
  - Number of outliers handled
- Duplicates removed: [number]
- Data type conversions: [list any conversions]
- Rows after cleaning: [number]

**Example format:**
```
DATA CLEANING REPORT
====================

Rows before cleaning: 50000

Missing Data Handling:
- Water Temperature: 2500 missing values (5.0%)
  Method: Forward-fill (time series appropriate)
  Result: All missing values filled
  
- Air Temperature: 1500 missing values (3.0%)
  Method: Forward-fill, then median imputation for remaining
  Result: All missing values filled

Outlier Handling:
- Water Temperature: Detected 500 outliers using IQR method (3×IQR)
  Method: Capped at bounds [Q1 - 3×IQR, Q3 + 3×IQR]
  Bounds: [-5.2, 35.8]
  Result: 500 values capped

Duplicates Removed: 0

Data Type Conversions:
- Measurement Timestamp: Converted to datetime64[ns]

Rows after cleaning: 50000
```

### 3. `output/q2_rows_cleaned.txt`
**Format:** Plain text file
**Content:** Single integer number (total rows after cleaning)
**Requirements:**
- Only the number, no text, no labels
- No whitespace before or after
- Example: `50000`

---

## Requirements Checklist

- [ ] Missing data handling strategy chosen and implemented
- [ ] Outliers detected and handled (IQR method, z-scores, or domain knowledge)
- [ ] Data types validated and converted
- [ ] Duplicates identified and removed
- [ ] Cleaning decisions documented in report
- [ ] All 3 required artifacts saved with exact filenames

---

## Your Approach

1. **Handle missing data** - Choose appropriate strategy (drop, forward-fill, impute) based on data characteristics
2. **Detect and handle outliers** - Use IQR method or z-scores; decide whether to remove, cap, or transform
3. **Validate data types** - Ensure numeric and datetime columns are properly typed
4. **Remove duplicates**
5. **Document and save** - Write detailed cleaning report explaining your decisions

---

## Decision Points

- **Missing data:** Should you drop rows, impute values, or forward-fill? Consider: How much data is missing? Is it random or systematic? For time series, forward-fill is often appropriate.
- **Outliers:** Are they errors or valid extreme values? Use IQR method or z-scores to detect, then decide: remove, cap, or transform. Document your reasoning.
- **Data types:** Are numeric columns actually numeric? Are datetime columns properly formatted? Convert as needed.

---

## Checkpoint

After Q2, you should have:
- [ ] Missing data handled
- [ ] Outliers addressed
- [ ] Data types validated
- [ ] Duplicates removed
- [ ] All 3 artifacts saved: `q2_cleaned_data.csv`, `q2_cleaning_report.txt`, `q2_rows_cleaned.txt`

---

**Next:** Continue to `q3_data_wrangling.md` for Data Wrangling.


## Start 

### Missing Data Handling

In [2]:
# Check data type
display(df.dtypes)

# Convert Measurement timestamp to datetime and set as index
df['Measurement Timestamp'] = pd.to_datetime(df['Measurement Timestamp'])
df = df.set_index('Measurement Timestamp')

# Verify datetime index
print("Index data type:", df.index.dtype)
df = df.sort_index()

Station Name                    object
Measurement Timestamp           object
Air Temperature                float64
Wet Bulb Temperature           float64
Humidity                         int64
Rain Intensity                 float64
Interval Rain                  float64
Total Rain                     float64
Precipitation Type             float64
Wind Direction                   int64
Wind Speed                     float64
Maximum Wind Speed             float64
Barometric Pressure            float64
Solar Radiation                  int64
Heading                        float64
Battery Life                   float64
Measurement Timestamp Label     object
Measurement ID                  object
dtype: object

Index data type: datetime64[ns]


In [3]:
# Save info for later report
rows_before_cleaning = len(df)

# Handle Missing Data
print("Missing Data Analysis")

# Detailed missing data analysis with data types for context
missing_analysis = pd.DataFrame({
    'Column': df.columns,
    'Type': df.dtypes.astype(str).values,
    'Missing': df.isnull().sum().values,
    'Missing %': (df.isnull().sum() / len(df) * 100).round(2).values
})
missing_analysis = missing_analysis[missing_analysis['Missing'] > 0].sort_values('Missing', ascending=False)

if len(missing_analysis) == 0:
    print("No missing values found")
else:
    display(missing_analysis)

# Check if missing values occur in the same rows
cols_with_same_missing = ['Wet Bulb Temperature', 'Rain Intensity', 'Total Rain', 'Precipitation Type', 'Heading']
missing_mask = df[cols_with_same_missing].isnull()
print(f"Rows where ALL are missing: {missing_mask.all(axis=1).sum():,}")
print(f"Rows where ANY is missing: {missing_mask.any(axis=1).sum():,}")
# If these numbers match, missingness is perfectly correlated

# Investigate: What characterizes the rows with missing data?
cols_available = [c for c in cols_with_same_missing if c in df.columns]
missing_mask = df[cols_available].isnull()

rows_with_missing = df[missing_mask.all(axis=1)]
rows_complete = df[~missing_mask.any(axis=1)]

# Check station distribution - is it one station's data?
display("By Station:")
station_comparison = pd.DataFrame({
    'Missing rows %': rows_with_missing['Station Name'].value_counts(normalize=True).round(3) * 100,
    'Complete rows %': rows_complete['Station Name'].value_counts(normalize=True).round(3) * 100
})
display(station_comparison)

# Check time distribution - are missing rows from a specific period?
print("By Date Range:")
display(pd.DataFrame({
    'Metric': ['Earliest measurement', 'Latest measurement', 'Count'],
    'Missing rows': [
        rows_with_missing.index.min(), 
        rows_with_missing.index.max(),
        f"{len(rows_with_missing):,}"
    ],
    'Complete rows': [
        rows_complete.index.min(), 
        rows_complete.index.max(),
        f"{len(rows_complete):,}"
    ]
}))

# Check if missing rows cluster on specific dates
print("Missing rows by date (top 10 dates):")
missing_by_date = rows_with_missing.groupby([rows_with_missing.index.year, rows_with_missing.index.month]).size().sort_values(ascending=False)
display(missing_by_date.head(10).to_frame('missing_count'))

display(df.groupby('Station Name').size())


Missing Data Analysis


,Column,Type,Missing,Missing %
2,Wet Bulb Temperature,float64,75926,38.68
4,Rain Intensity,float64,75926,38.68
6,Total Rain,float64,75926,38.68
7,Precipitation Type,float64,75926,38.68
13,Heading,float64,75926,38.68
11,Barometric Pressure,float64,146,0.07
1,Air Temperature,float64,75,0.04


Rows where ALL are missing: 75,926
Rows where ANY is missing: 75,926


'By Station:'

,Missing rows %,Complete rows %
Station Name,,
63rd Street Weather Station,NaN,41.5
Foster Weather Station,100.0,NaN
Oak Street Weather Station,NaN,58.5


By Date Range:


,Metric,Missing rows,Complete rows
0,Earliest measurement,2015-05-22 16:00:00,2015-04-25 09:00:00
1,Latest measurement,2025-12-02 12:00:00,2025-12-02 12:00:00
2,Count,"75,926","120,345"


Missing rows by date (top 10 dates):


missing_count
Measurement Timestamp Measurement Timestamp               
2023                  1                                743
2022                  7                                743
2019                  8                                742
2020                  10                               742
2025                  7                                741
                      5                                740
2022                  1                                739
                      8                                739
2019                  1                                739
2025                  10                               738

Station Name
63rd Street Weather Station    49951
Foster Weather Station         75926
Oak Street Weather Station     70394
dtype: int64

In [4]:
# Strategy for handling missing data
# After investigation, I realized all the missing data from 'Foster Weather Station'.
# Specifically, the station is missing values in key raining data such as total rain
# and precipitation type. Since measurements can vary significantly between stations 
# due to microclimates or other local factors, I don't think it is a good idea to fill
# the missing data using values from other weather station. Instead, I think it's best 
# that we drop the data from Foster Weather Station from the analysis.
df_no_missing = df[df['Station Name'].isin(['63rd Street Weather Station', 'Oak Street Weather Station'])]
# df_no_missing['Rain Intensity'] = df_no_missing.fillna(df_no_missing.groupby(df_no_missing.index))

# For the remaining missing value in column Air Temperature and Barometric Pressure,
# I tried to fill missing value using forward filling within each station. However, 
# the method failed because the missing value is at the start of the data. Therefore,
# I use backword filling instead to fill the missing value while retaining the trend
# and distribution.
df_no_missing['Air Temperature'] = df_no_missing.groupby('Station Name')['Air Temperature'].bfill()
df_no_missing['Barometric Pressure'] = df_no_missing.groupby('Station Name')['Barometric Pressure'].bfill()

# Check dataset shape after dropping to ensure we are have enough data
print("Data shape after dropping missing value:", df_no_missing.shape)

# Check if all missing values are filled
missing_analysis = pd.DataFrame({
    'Column': df_no_missing.columns,
    'Type': df_no_missing.dtypes.astype(str).values,
    'Missing': df_no_missing.isnull().sum().values,
    'Missing %': (df_no_missing.isnull().sum() / len(df_no_missing) * 100).round(2).values
})
missing_analysis = missing_analysis[missing_analysis['Missing'] > 0].sort_values('Missing', ascending=False)

if len(missing_analysis) == 0:
    print("No missing values found")
else:
    print("Failed to handle missing data")
    display(missing_analysis)

Data shape after dropping missing value: (120345, 17)
No missing values found


/var/folders/5t/lfb7y4m975b78bly378zr4880000gn/T/ipykernel_32473/2617075055.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_no_missing['Air Temperature'] = df_no_missing.groupby('Station Name')['Air Temperature'].bfill()
/var/folders/5t/lfb7y4m975b78bly378zr4880000gn/T/ipykernel_32473/2617075055.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_no_missing['Barometric Pressure'] = df_no_missing.groupby('Station Name')['Barometric Pressure'].bfill()


### Outlier Handling

In [5]:
# Adopted the 'detect_outliers_iqr' function from demo
# Identify outliers using IQR method with Tukey fences
def detect_outliers_iqr(df, column, iqr_multiplier=1.5):
    """
    Detect outliers using the IQR (Interquartile Range) method.

    This is the Tukey fence method, which is robust and doesn't assume
    normal distribution. It's widely used in exploratory data analysis.

    Parameters:
    -----------
    df : DataFrame
        The data
    column : str
        Column name to check for outliers
    iqr_multiplier : float, default=1.5
        Tukey fence multiplier:
        - 1.5 = standard outlier detection (common choice)
        - 3.0 = extreme outlier detection (more conservative)

    Returns:
    --------
    tuple : (outliers DataFrame, lower_bound, upper_bound)
    """
    Q1 = df[column].quantile(0.25)  # 25th percentile
    Q3 = df[column].quantile(0.75)  # 75th percentile
    IQR = Q3 - Q1  # Interquartile range (middle 50% of data)

    # Tukey fences: standard statistical method for outlier detection
    lower_bound = Q1 - iqr_multiplier * IQR
    upper_bound = Q3 + iqr_multiplier * IQR

    # Find values outside the fences
    outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]
    return outliers, lower_bound, upper_bound

In [6]:
# Select numeric columns only
numeric_cols = df_no_missing.select_dtypes(include=[np.number]).columns.tolist()

# Check outliers for all numerical columns using IQR method 
# for a general understanding
outlier_df = pd.DataFrame(columns= ['Column Name', 'Lower bound', 'Upper bound', 'Number of outliers', 'Outlier %'])
for col in numeric_cols:
    col_outliers, col_lower, col_upper = detect_outliers_iqr(df_no_missing, col)
    outlier_df.loc[len(outlier_df)] = [col, col_lower, col_upper, len(col_outliers), round((len(col_outliers)/len(df_no_missing)*100),2)]

outlier_df = outlier_df.set_index('Column Name')
print("Data outlier summary:")
display(outlier_df)

Data outlier summary:


,Lower bound,Upper bound,Number of outliers,Outlier %
Column Name,,,,
Air Temperature,-20.50,47.50,59,0.05
Wet Bulb Temperature,-20.10,41.50,88,0.07
Humidity,24.50,116.50,160,0.13
Rain Intensity,0.00,0.00,4243,3.53
Interval Rain,0.00,0.00,9383,7.80
Total Rain,-259.00,476.20,9507,7.90
Precipitation Type,0.00,0.00,8414,6.99
Wind Direction,-239.50,580.50,0,0.00
Wind Speed,-2.15,7.05,3945,3.28


In [7]:
# Handle outliers based on domain knowledge
print("=== Handling Outliers ===")

# Instead of dropping rows, we'll create an EXCLUSION FLAG
# This approach is better because:
# 1. Preserves original data for analysis
# 2. Transparent about what's being filtered
# 3. Can toggle exclusions on/off
# 4. Easy to see WHY a row was excluded

df_clean = df_no_missing.copy()
df_clean['exclude'] = False  # Start with no exclusions
df_clean['exclude_reason'] = ''  # Track why rows are excluded

print(f"Original shape: {df_clean.shape[0]:,} rows × {df_clean.shape[1]} columns")

# Define some data quality thresholds based on domain knowledge
MIN_HUMIDITY = 0  # Humidity cannot be negative
# MIN_RAIN_INTENSITY = 0  # Rain cannot be negative
# MIN_INTERVAL_RAIN = 0  # Rain cannot be negative
# MIN_TOTAL_RAIN = 0  # Rain cannot be negative
MIN_WIND_SPEED = 0
MAX_WIND_SPEED = 103.3 # Maximum wind speed ever recorded in the US
MIN_BAROMETRIC_PRESSURE = 870 # The lowest non-tornadic pressure ever recorded
MAX_BAROMETRIC_PRESSURE = 1083.8 #  The highest sea-level air pressure ever officially recorded on Earth was 1083.8 hPa in Agata, Siberia, in 1968. 
MIN_SOLAR_RADIATION = 0
MIN_HEADING = 0
MAX_HEADING = 359
EXPECTED_TYPE = [0, 40, 60, 70]

# Flag exclusions with reasons (using bitwise OR to accumulate flags)
def flag_exclusion(df, condition, reason):
    """Flag rows for exclusion and record the reason."""
    mask = condition
    df.loc[mask & ~df['exclude'], 'exclude_reason'] = reason  # First reason wins
    df.loc[mask, 'exclude'] = True
    return mask.sum()

# Apply exclusion rules, either with IQR method or domain knowledge
n_outlier_air_temp = flag_exclusion(df_clean, ~df_clean['Air Temperature'].between(outlier_df.loc['Air Temperature', 'Lower bound'], outlier_df.loc['Air Temperature', 'Upper bound']), 'Air Temperature > or < 1.5*IQR')
n_outlier_wetbulb_temp = flag_exclusion(df_clean, ~df_clean['Wet Bulb Temperature'].between(outlier_df.loc['Wet Bulb Temperature', 'Lower bound'], outlier_df.loc['Wet Bulb Temperature', 'Upper bound']), 'Wet bulb Temp > or < 1.5*IQR')
n_outlier_humidity = flag_exclusion(df_clean, ~df_clean['Humidity'].between(outlier_df.loc['Humidity', 'Lower bound'], outlier_df.loc['Humidity', 'Upper bound']), 'Humidity > or < 1.5*IQR')
# n_outlier_rain = flag_exclusion(df_clean, ~df_clean['Total Rain'].between(outlier_df.loc['Total Rain', 'Lower bound'], outlier_df.loc['Total Rain', 'Upper bound']), 'Humidity > or < 1.5*IQR')
n_outlier_wind = flag_exclusion(df_clean, ~df_clean['Wind Speed'].between(MIN_WIND_SPEED, MAX_WIND_SPEED), 'negative_wind_speed or > max_wind_speed')
n_outlier_max_wind = flag_exclusion(df_clean, ~df_clean['Maximum Wind Speed'].between(MIN_WIND_SPEED, MAX_WIND_SPEED), 'negative_wind_speed or > max_wind_speed')
n_outlier_pressure = flag_exclusion(df_clean, ~df_clean['Barometric Pressure'].between(MIN_BAROMETRIC_PRESSURE, MAX_BAROMETRIC_PRESSURE), 'unreasonable_pressure')
n_outlier_solar = flag_exclusion(df_clean, df_clean['Solar Radiation'] < 0, 'negative_solar_radiation')
n_outlier_battery = flag_exclusion(df_clean, ~df_clean['Battery Life'].between(outlier_df.loc['Battery Life', 'Lower bound'], outlier_df.loc['Battery Life', 'Upper bound']), 'Humidity > or < 1.5*IQR')

# display(df_clean[ df_clean['Solar Radiation'] < -6][['Solar Radiation', 'exclude']].head(10))
# print(n_outlier_solar)
# Move forward with cleaned data only
df_valid = df_clean[~df_clean['exclude']]
print(f"Shape after outlier handling: {df_valid.shape[0]:,} rows × {df_valid.shape[1]} columns")

=== Handling Outliers ===
Original shape: 120,345 rows × 19 columns
Shape after outlier handling: 103,559 rows × 19 columns


### Duplicate Detection

In [8]:
# Check for duplicate rows
print("=== Duplicate Detection ===")

# Check for completely duplicate rows
# n_duplicates = df_clean.duplicated().sum()
# print(f"Completely duplicate rows: {n_duplicates:,}")

# Check for completely duplicate rows and mark duplicates
duplicates = df_clean.duplicated(keep='first')
n_duplicates = flag_exclusion(df_clean, duplicates, 'duplicate')
print(f"Completely duplicate rows: {n_duplicates:,}")

# Move forward with cleaned data only
df_valid = df_clean[~df_clean['exclude']]
print(f"Shape after duplicate handling: {df_valid.shape[0]:,} rows × {df_valid.shape[1]} columns")

# Overall summary
n_excluded = df_clean['exclude'].sum()
display(pd.DataFrame({
    'Metric': ['Total rows', 'Rows to keep', 'Rows excluded'],
    'Value': [
        f"{len(df_clean):,}",
        f"{(~df_clean['exclude']).sum():,}",
        f"{n_excluded:,} ({n_excluded/len(df_clean)*100:.2f}%)"
    ]
}))

=== Duplicate Detection ===
Completely duplicate rows: 0
Shape after duplicate handling: 103,559 rows × 19 columns


,Metric,Value
0,Total rows,"120,345"
1,Rows to keep,"103,559"
2,Rows excluded,"16,786 (13.95%)"


### Data Type Validation and Conversion

In [9]:
# Ensure numeric columns are proper types
numeric_cols = ['Air Temperature', 'Wet Bulb Temperature', 'Humidity', 'Rain Intensity', 
                'Interval Rain', 'Total Rain', 'Wind Direction', 'Wind Speed',
                'Maximum Wind Speed', 'Barometric Pressure', 'Solar Radiation',
                'Heading']
for col in numeric_cols:
    df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce')

### Cleaning Report

In [10]:
# Save cleaned data
df_valid.to_csv('output/q2_cleaned_data.csv')

# Write cleaning report
with open('output/q2_cleaning_report.txt', 'w') as f:
    f.write('DATA CLEANING REPORT\n')
    f.write('====================\n')
    f.write(f'\nRows before cleaning: {rows_before_cleaning}\n')

    f.write(f'\nMissing Data Handling:\n')

    f.write(f'\nOutlier Handling:\n')
    
    f.write(f'\nDuplicates Removed: {n_duplicates}\n')

    f.write(f'\nData Type Conversions:\n')
    f.write(f'- Measurement Timestamp: Converted to {df_valid.index.dtype}')

    f.write(f'\nRows after cleaning: {len(df_valid)}\n')
# Rows cleaned
with open('output/q2_rows_cleaned.txt', 'w') as f:
    f.write(f"{len(df_valid)}")